In [37]:
import gurobipy as gp
from gurobipy import *
import numpy as np


capacity = [5,8,7]
demand = [3,6,6,5]
varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]
locations = 3
customers = 4


"""
file = open('43.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]

fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]
"""

"\nfile = open('43.txt','r')\ndata = file.readlines()\n\n[locations, customers] = [int(x) for x in data[0].split(' ')]\ncapacity = [int(data[1].split(' ')[i]) for i in range(locations) ]\ndemand = [int(data[2].split(' ')[j]) for j in range(customers) ]\n\nfixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]\nvarcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]\n"

In [38]:
####
#   Benders decomposition via Gurobi + Python
#   Example 3.1 from Conejo et al.'s book on optimization techniques
####

##
# To Run:
# m = Benders_Master()
# m.optimize()
##


# Class which can have attributes set.
class expando(object):
    pass


In [39]:
# Master problem
class Benders_Master:
    global locations
    global customers
    global demand
    global capacity
    global fixcost
    global varcost
    
    def __init__(self, benders_gap=0.1):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self._load_data(benders_gap=benders_gap)
        self._build_model()
        
        
    def optimize(self, simple_results=False):
        # Initial solution
        self.model.optimize()
        
        # Build subproblem from solution
        self.submodel = Benders_Subproblem(self)
        self.submodel.update_fixed_vars(self)
        self.submodel.optimize()
        
        self.infsubmodel = Benders_InfSubproblem(self)
        self.infsubmodel.update_fixed_vars(self)
        self.infsubmodel.optimize() 
        
        self._add_cut()
        self._update_bounds()
        self._save_vars()
        
        #while self.data.ub > self.data.lb + self.data.benders_gap and len(self.data.cutlist) < self.max_iters:
        while self.data.ub > self.data.lb + self.data.benders_gap:
            self.model.optimize()
            
            self.submodel.update_fixed_vars(self)
            self.submodel.optimize()
            
            if self.submodel.model.Status != 2:
                #self.infsubmodel = Benders_InfSubproblem(self)
                self.infsubmodel.update_fixed_vars(self)
                self.infsubmodel.optimize()            
            
            self._add_cut()
            self._update_bounds()
            self._save_vars()
        pass
    
        ####   Loading functions
    ###

    def _load_data(self, benders_gap=0.1):
        self.data.optcutlist = []
        self.data.fcutlist = []
        
        self.data.upper_bounds = []
        self.data.lower_bounds = []
        self.data.benders_gap = benders_gap
        self.data.ub = gp.GRB.INFINITY
        self.data.lb = -gp.GRB.INFINITY
        
        self.data.xs = []
        self.data.ys = []
        self.data.zs = []
    
    
    ###
    #   Model Building
    ###
    def _build_model(self):
        self.model = gp.Model()
        self._build_variables()
        self._build_constraints()
        self.model.update()

    def _build_variables(self):
        m = self.model
        
        self.variables.y = {}
        for i in range(locations):
            for j in range(customers):     
                self.variables.y[i,j] = m.addVar(vtype=GRB.BINARY) #### integer variable y
        
        self.variables.z = m.addVar(obj = 1) # z from subproblem
        m.update()
        

    def _build_constraints(self):
        self.constraints.cuts = {}
        pass
    

    ###
    # Cut adding
    ###
    def _add_cut(self):
        y = self.variables.y
        cut = len(self.data.optcutlist) + len(self.data.fcutlist)
        
        if self.submodel.model.Status not in [3,4,5]:        
            self.data.optcutlist.append(len(self.data.optcutlist))

            # Get dual variables from subproblem
            v = np.zeros(customers) #dual of demand constraints
            for j in range(customers):
                v[j] = self.submodel.constraints.demand_constr[j].pi #get dual value

            w = np.zeros((locations,customers)) #dual of logical constraints
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = self.submodel.constraints.log_constr[i,j].pi

            u = np.zeros(locations) #dual of capacity constraints
            for i in range(locations):
                u[i] = self.submodel.constraints.cap_constr[i].pi
            
            #z_sub = self.submodel.model.ObjVal
            
            # Generate cut
            self.constraints.cuts[cut] = self.model.addConstr(
                self.variables.z, gp.GRB.GREATER_EQUAL, 
                sum(demand[j] * v[j] for j in range(customers)) - 
                sum(capacity[i] * u[i] for i in range(locations)) - 
                sum(min(capacity[i], demand[j])*w[i,j]*y[i,j] for i in range(locations) for j in range(customers)) + 
            sum(fixcost[i][j]*y[i,j] for i in range(locations) for j in range(customers))) 
            
            
        else:
            self.data.fcutlist.append(len(self.data.fcutlist))
            fv = self.infsubmodel.variables.fv
            fu = self.infsubmodel.variables.fu
            fw = self.infsubmodel.variables.fw

            self.constraints.cuts[cut] = self.model.addConstr(
                sum(demand[j] * fv[j].x for j in range(customers)) - 
                sum(capacity[i] * fu[i].x for i in range(locations)) - 
                sum(min(capacity[i], demand[j])*fw[i,j].x*y[i,j] for i in range(locations) for j in range(customers)), 
                gp.GRB.LESS_EQUAL,0)            
            
    

    ###
    # Update upper and lower bounds
    ###
    def _update_bounds(self):
        if self.submodel.model.Status == 2:
            z_sub = self.submodel.model.ObjVal
            #z_master = self.model.ObjVal
            fc = sum(fixcost[i][j]*self.variables.y[i,j].x for i in range(locations) for j in range(customers))
            self.data.ub = min(self.data.ub, fc + z_sub)
        
        # The best lower bound is the current bestbound,
        # This will equal z_master at optimality
        self.data.lb = self.model.ObjBound
        self.data.upper_bounds.append(self.data.ub)
        self.data.lower_bounds.append(self.data.lb)

    def _save_vars(self):
        self.data.ys.append(self.variables.y)
        self.data.zs.append(self.variables.z)
        self.data.xs.append(self.submodel.variables.x)        



In [40]:
# Subproblem
class Benders_Subproblem:
    def __init__(self, MP):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self._build_model()
        self.data.MP = MP
        self.update_fixed_vars()

    def optimize(self):
        self.model.optimize()
        

    ###
    #   Model Building
    ###
    def _build_model(self):
        self.model = gp.Model()
        self._build_variables()
        self._build_constraints()
        self.model.update()

    def _build_variables(self):
        m = self.model

        # Power flow on line l
        self.variables.x = {}
        for i in range(locations):
            for j in range(customers):
                self.variables.x[i,j] = m.addVar(obj = varcost[i][j]) #transportation qty

        self.variables.y_free = {}
        for i in range(locations):
            for j in range(customers):
                self.variables.y_free[i,j] = m.addVar() #decision
        m.update()

        
    def _build_constraints(self):
        m = self.model
        x = self.variables.x
        y = self.variables.y_free

        #demand constraint
        self.constraints.demand_constr = {} 
        for j in range(customers):
            self.constraints.demand_constr[j] = m.addConstr( sum(x[i,j] for i in range(locations)) , gp.GRB.GREATER_EQUAL, demand[j])

        #capacity constraint
        self.constraints.cap_constr = {}
        for i in range(locations):
            self.constraints.cap_constr[i] = m.addConstr(sum(x[i,j] for j in range(customers)), gp.GRB.LESS_EQUAL, capacity[i])
        
        #logical constraint
        self.constraints.log_constr = {}
        for i in range(locations):
            for j in range(customers):
                self.constraints.log_constr[i,j] = m.addConstr(x[i,j], gp.GRB.LESS_EQUAL, y[i,j] * min(capacity[i], demand[j]))
        
        self.constraints.fix_y = {}
        for i in range(locations):
            for j in range(customers):
                self.constraints.fix_y[i,j] = m.addConstr(y[i,j], gp.GRB.EQUAL, 1)

    def update_fixed_vars(self, MP=None):
        if MP is None:
            MP = self.data.MP
        for i in range(locations):
            for j in range(customers):     
                self.constraints.fix_y[i,j].rhs = MP.variables.y[i,j].x


In [41]:
# Subproblem
class Benders_InfSubproblem:
    def __init__(self, MP):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self._build_model()
        self.data.MP = MP
        self.update_fixed_vars()

    def optimize(self):
        self.model.modelSense = -1
        self.model.optimize()

    ###
    #   Model Building
    ###
    def _build_model(self):
        self.model = gp.Model()
        self._build_variables()
        self._build_constraints()
        self.model.update()

    def _build_variables(self):
        m = self.model

        self.variables.dummy = m.addVar(obj = 1)

        self.variables.fv = {}
        for j in range(customers): self.variables.fv[j] = m.addVar()
        self.variables.fw = {}
        for i in range(locations):
            for j in range(customers):
                self.variables.fw[i,j] = m.addVar()
        self.variables.fu = {}
        for i in range(locations): self.variables.fu[i] = m.addVar()

        self.variables.y_free = {}
        for i in range(locations):
            for j in range(customers):
                self.variables.y_free[i,j] = m.addVar() #decision
        m.update()                 

        
    def _build_constraints(self):
        m = self.model
        fv = self.variables.fv
        fu = self.variables.fu
        fw = self.variables.fw
        y = self.variables.y_free
        
        self.constraints.c1 =  m.addConstr( sum(demand[j]*fv[j] for j in range(customers)) +  
                          sum(- capacity[i]*fu[i] for i in range(locations)) +  
                          sum(- min(capacity[i], demand[j])*y[i,j]*fw[i,j] for i in range(locations) for j in range(customers)), gp.GRB.EQUAL, 1)
        
        self.constraints.c2 = {}
        for i in range(locations):
            for j in range(customers):
                self.constraints.c2[i,j] =  m.addConstr( fv[j] - fu[i] - fw[i,j], gp.GRB.LESS_EQUAL, 0);        
        
        self.constraints.c3 =  m.addConstr(self.variables.dummy, gp.GRB.EQUAL, 0)

        self.constraints.fix_y = {}
        for i in range(locations):
            for j in range(customers):
                self.constraints.fix_y[i,j] = m.addConstr(y[i,j], gp.GRB.EQUAL, 0)

    def update_fixed_vars(self, MP=None):
        if MP is None:
            MP = self.data.MP
        for i in range(locations):
            for j in range(customers):     
                self.constraints.fix_y[i,j].rhs = MP.variables.y[i,j].x

In [42]:
m = Benders_Master()
m.optimize()


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 0 rows, 13 columns and 0 nonzeros
Model fingerprint: 0x83e403fd
Variable types: 1 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 31 rows, 24 columns and 60 nonzeros
Model fingerprint: 0xcd87b9d1
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 24 rows and 24 columns
Presolve

  QRHS range       [1e+00, 1e+00]
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 13 rows, 19 columns, 46 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.200e+01
 Factor NZ  : 9.100e+01
 Factor Ops : 8.190e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -0.00000000e+00 -0.00000000e+00  2.39e-01 0.00e+00  9.68e-03     0s
   1  -0.00000000e+00 -1.17398931e-02  2.78e-17 1.74e-02  1.83e-03     0s
   2  -0.00000000e+00  2.73365004e-07  5.00e-16 2.01e-06  2.19e-07     0s
   3  -0.00000000e+00  2.73365031e-13  3.61e-16 2.01e-12  2.20e-13     0s

Barrier solved model in 3 iterations and 0.01 seconds
Optimal objective -0.00000000e+00

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 4 rows, 13 columns and 48 nonzeros
Model fingerprint: 0x56ec4e14
Variable types: 1 continuous, 12 integer 

H    0     0                     137.0000000  136.55299  0.33%     -    0s
     0     0  136.55299    0    3  137.00000  136.55299  0.33%     -    0s

Explored 1 nodes (4 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 137 140 148 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.370000000000e+02, best bound 1.370000000000e+02, gap 0.0000%
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 31 rows, 24 columns and 60 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1100000e+02   1.500000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Infeasible model
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 25 rows, 32 columns and 49 nonzeros
Model fingerprint: 

   3  -0.00000000e+00 -6.15798314e-10  6.66e-16 1.18e-09  1.48e-10     0s

Barrier solved model in 3 iterations and 0.02 seconds
Optimal objective -0.00000000e+00

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 9 rows, 13 columns and 108 nonzeros
Model fingerprint: 0x4267600b
Variable types: 1 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R8 by 1.000000000

Found heuristic solution: objective 148.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 70 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 1.377287e+02, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |   


Solved in 3 iterations and 0.00 seconds
Infeasible model
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 25 rows, 32 columns and 49 nonzeros
Model fingerprint: 0x7912d740
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+00, 6e+00]
  QLMatrix range   [3e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 13 rows and 13 columns
Presolve time: 0.01s
Presolved: 13 rows, 19 columns, 49 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.200e+01
 Factor NZ  : 9.100e+01
 Factor Ops : 8.190e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -0.00000000e+00 -0.00000000e+00  3.70e-01 0.00e+00  8.52e-03     0s
   1  -0.00000000e+00 -2.30122943

In [43]:
for i in m.data.ys[-1].keys():
    print(m.data.ys[-1][i].x)

temp = [m.data.ys[-1][i].x for i in m.data.ys[-1].keys()]
k = 0
y0=np.zeros((locations, customers))
for i in range(locations):
    for j in range(customers):
        y0[i,j] = temp[k]
        k+=1
y0

1.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0


array([[1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 1., 1., 0.]])

In [53]:
y0 = [[1,0,0,1],[0,0,0,1],[1,1,1,0]]
y0

[[1, 0, 0, 1], [0, 0, 0, 1], [1, 1, 1, 0]]

In [52]:
model0 = gp.Model()
x0 = {}
for i in range(locations):
    for j in range(customers):
        x0[i,j] = model0.addVar(obj=varcost[i][j])

#demand constraint
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) >= demand[j])
        
#capacity constraint
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

#logical constraint
log_constr = {}
for i in range(locations):
    for j in range(customers):
        log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i][j] * min(capacity[i], demand[j]))

model0.optimize()

print("Objective: " + str(model0.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x0[i,j].x

print(solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 19 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xe88833da
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 15 rows and 8 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


AttributeError: Unable to retrieve attribute 'objVal'

In [48]:
m.submodel.variables.x

{(0, 0): <gurobi.Var C0 (value 0.0)>,
 (0, 1): <gurobi.Var C1 (value 0.0)>,
 (0, 2): <gurobi.Var C2 (value 0.0)>,
 (0, 3): <gurobi.Var C3 (value 5.0)>,
 (1, 0): <gurobi.Var C4 (value 0.0)>,
 (1, 1): <gurobi.Var C5 (value 6.0)>,
 (1, 2): <gurobi.Var C6 (value 2.0)>,
 (1, 3): <gurobi.Var C7 (value 0.0)>,
 (2, 0): <gurobi.Var C8 (value 3.0)>,
 (2, 1): <gurobi.Var C9 (value 0.0)>,
 (2, 2): <gurobi.Var C10 (value 4.0)>,
 (2, 3): <gurobi.Var C11 (value 0.0)>}

In [ ]:
    infbsp = gp.Model('Infeasible subproblem')
    dummy = infbsp.addVar(obj = 1)

    v = {}
    for j in range(customers): v[j] = infbsp.addVar()
    w = {}
    for i in range(locations):
        for j in range(customers):
            w[i,j] = infbsp.addVar()
    u = {}
    for i in range(locations): u[i] = infbsp.addVar()

    infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) +  
                      sum(- capacity[i]*u[i] for i in range(locations)) +  
                      sum(- bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) == 1)                        
    for i in range(locations):
        for j in range(customers):
            infbsp.addConstr( v[j] - u[i] - w[i,j] <= 0);
    infbsp.addConstr(dummy == 0)

    print(infbsp.Status)

    infbsp.modelSense = -1 #set to maximization
    infbsp.optimize()       

In [ ]:
# Import Gurobi Library
import gurobipy as gb

####
#   Benders decomposition via Gurobi + Python
#   Example 3.1 from Conejo et al.'s book on optimization techniques
####

##
# To Run:
# m = Benders_Master()
# m.optimize()
##


# Class which can have attributes set.
class expando(object):
    pass


# Master problem
class Benders_Master:
    def __init__(self, benders_gap=0.001, max_iters=10):
        self.max_iters = max_iters
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self._load_data(benders_gap=benders_gap)
        self._build_model()

    def optimize(self, simple_results=False):
        # Initial solution
        self.model.optimize()
        # Build subproblem from solution
        self.submodel = Benders_Subproblem(self)
        self.submodel.update_fixed_vars(self)
        self.submodel.optimize()
        self._add_cut()
        self._update_bounds()
        self._save_vars()
        while self.data.ub > self.data.lb + self.data.benders_gap and len(self.data.cutlist) < self.max_iters:
            self.model.optimize()
            self.submodel.update_fixed_vars(self)
            self.submodel.optimize()
            self._add_cut()
            self._update_bounds()
            self._save_vars()
        pass

    ###
    #   Loading functions
    ###

    def _load_data(self, benders_gap=0.001):
        self.data.cutlist = []
        self.data.upper_bounds = []
        self.data.lower_bounds = []
        self.data.lambdas = {}
        self.data.benders_gap = benders_gap
        self.data.ub = gb.GRB.INFINITY
        self.data.lb = -gb.GRB.INFINITY
        self.data.xs = []
        self.data.ys = []
        self.data.alphas = []

    ###
    #   Model Building
    ###
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()

    def _build_variables(self):
        m = self.model

        self.variables.x = m.addVar(lb=0.0, ub=16.0, name='x')
        self.variables.alpha = m.addVar(lb=-25.0, ub=gb.GRB.INFINITY, name='alpha')
        m.update()

    def _build_objective(self):
        self.model.setObjective(
            -self.variables.x/4 + self.variables.alpha,
            gb.GRB.MINIMIZE)

    def _build_constraints(self):
        self.constraints.cuts = {}
        pass

    ###
    # Cut adding
    ###
    def _add_cut(self):
        x = self.variables.x
        cut = len(self.data.cutlist)
        self.data.cutlist.append(cut)
        # Get sensitivity from subproblem
        sens = self.submodel.constraints.fix_x.pi
        z_sub = self.submodel.model.ObjVal
        # Generate cut
        self.constraints.cuts[cut] = self.model.addConstr(
            self.variables.alpha,
            gb.GRB.GREATER_EQUAL,
            z_sub + sens * (x - x.x))

    ###
    # Update upper and lower bounds
    ###
    def _update_bounds(self):
        z_sub = self.submodel.model.ObjVal
        z_master = self.model.ObjVal
        self.data.ub = z_master - self.variables.alpha.x + z_sub
        # The best lower bound is the current bestbound,
        # This will equal z_master at optimality
        self.data.lb = self.model.ObjBound
        self.data.upper_bounds.append(self.data.ub)
        self.data.lower_bounds.append(self.data.lb)

    def _save_vars(self):
        self.data.xs.append(self.variables.x.x)
        self.data.ys.append(self.submodel.variables.y.x)
        self.data.alphas.append(self.variables.alpha.x)


# Subproblem
class Benders_Subproblem:
    def __init__(self, MP):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self._build_model()
        self.data.MP = MP
        self.update_fixed_vars()

    def optimize(self):
        self.model.optimize()

    ###
    #   Model Building
    ###
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()

    def _build_variables(self):
        m = self.model

        # Power flow on line l
        self.variables.y = m.addVar(lb=0.0, ub=gb.GRB.INFINITY, name='y')
        self.variables.x_free = m.addVar(lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name='x_free')

        m.update()

    def _build_objective(self):
        m = self.model

        self.model.setObjective(
            - self.variables.y,
            gb.GRB.MINIMIZE)

    def _build_constraints(self):
        m = self.model
        y = self.variables.y
        x = self.variables.x_free

        self.constraints.c1 = m.addConstr(y - x, gb.GRB.LESS_EQUAL, 5.)
        self.constraints.c2 = m.addConstr(y - x/2., gb.GRB.LESS_EQUAL, 15./2.)
        self.constraints.c3 = m.addConstr(y + x/2., gb.GRB.LESS_EQUAL, 35./2.)
        self.constraints.c4 = m.addConstr(-y + x, gb.GRB.LESS_EQUAL, 10.)
        self.constraints.fix_x = m.addConstr(x, gb.GRB.EQUAL, 0.)

    def update_fixed_vars(self, MP=None):
        if MP is None:
            MP = self.data.MP
        self.constraints.fix_x.rhs = MP.variables.x.x